In [57]:
import pandas as pd

df = pd.read_csv('Fraud_Data.csv', parse_dates=['purchase_time', 'signup_time'])

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   user_id         151112 non-null  int64         
 1   signup_time     151112 non-null  datetime64[ns]
 2   purchase_time   151112 non-null  datetime64[ns]
 3   purchase_value  151112 non-null  int64         
 4   device_id       151112 non-null  object        
 5   source          151112 non-null  object        
 6   browser         151112 non-null  object        
 7   sex             151112 non-null  object        
 8   age             151112 non-null  int64         
 9   ip_address      151112 non-null  float64       
 10  class           151112 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(4), object(4)
memory usage: 12.7+ MB


In [7]:
df.isna().values.any()

False

In [9]:
# no missing values
# if there were, I would either:
# 1. drop the column
# 2. drop those records
# 3. replace those missing values

In [8]:
df.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [ ]:
# which columns do I think are not going to be needed
# 1. user_id


In [11]:
df['user_id'].nunique() == df.shape[0]

True

In [12]:
df['device_id'].nunique()

137956

In [13]:
# Device ID are not all unique
# so maybe I can use it for predictions
# for example, create a feature about if they are a repeat device

In [14]:
df['source'].unique()

array(['SEO', 'Ads', 'Direct'], dtype=object)

In [15]:
df['browser'].unique()

array(['Chrome', 'Opera', 'Safari', 'IE', 'FireFox'], dtype=object)

In [16]:
df['sex'].unique()

array(['M', 'F'], dtype=object)

In [20]:
df['ip_address'].nunique()

143512

In [21]:
# For IP address
# maybe consider if we've seen them before
# IP address does not really seem to be like they are IP addresses
# Use IP addres to determine country of origin

In [26]:
df['ip_address'].iloc[3]

3840542443.91396

In [60]:
# repeat device appears more than once
# so I need to determine how many times each device ID appears
# in the data set

df_out = (
    df
    .set_index('device_id')
    .assign(repeat=lambda df: df.index.value_counts() > 1)
    .reset_index()
)

In [37]:
# Well, if you are repeat device, are you more likely to commit fraud?

In [41]:
# for each category of repeat, find the fraction that are fraud
# those that are fraud have a class value of 1

df_out['class'].mean()

0.09364577267192546

In [45]:
df_out.groupby('repeat').agg(mean=('class', 'mean'), count=('class', 'count'))

,mean,count
repeat,,
False,0.030429,131781
True,0.524598,19331


In [46]:
# So what I've learned is that there's a correlation between repeate device and fraud
# I should probably include such feature.
# That feature could be as simple as True/False or continuous (the number of occurences)

In [47]:
df_out.groupby('class')['purchase_value'].mean()

class
0    36.929418
1    36.993004
Name: purchase_value, dtype: float64

In [49]:
df_out.groupby('class')['age'].mean()

class
0    33.122356
1    33.318281
Name: age, dtype: float64

In [50]:
df_out.groupby('source')['class'].mean()

source
Ads       0.092066
Direct    0.105370
SEO       0.089285
Name: class, dtype: float64

In [51]:
df_out.groupby('sex')['class'].mean()

sex
F    0.091007
M    0.095523
Name: class, dtype: float64

In [52]:
df_out.groupby('browser')['class'].mean()

browser
Chrome     0.098792
FireFox    0.095165
IE         0.086775
Opera      0.089227
Safari     0.090201
Name: class, dtype: float64

In [62]:
df_out.assign(time_diff=lambda df: (df['purchase_time'] - df['signup_time']))

,device_id,user_id,signup_time,purchase_time,purchase_value,source,browser,sex,age,ip_address,class,repeat,time_diff
0,QVPSPJUOCKZAR,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,SEO,Chrome,M,39,7.327584e+08,0,False,52 days 03:51:22
1,EOGFQPIZPYXFZ,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,Ads,Chrome,F,53,3.503114e+08,0,False,0 days 04:59:04
2,YSSKYOSJHPPLJ,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,SEO,Opera,M,53,2.621474e+09,1,True,0 days 00:00:01
3,ATGTXKYKUDUQN,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,SEO,Safari,M,41,3.840542e+09,0,False,5 days 16:41:25
4,NAUITBZFJKHWW,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,Ads,Safari,M,45,4.155831e+08,0,False,50 days 11:31:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151107,XPSKTWGPWINLR,345170,2015-01-27 03:03:34,2015-03-29 00:30:47,43,SEO,Chrome,M,28,3.451155e+09,1,True,60 days 21:27:13
151108,LYSFABUCPCGBA,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,SEO,Safari,M,32,2.439047e+09,0,False,10 days 18:41:10
151109,MEQHCSJUBRBFE,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,SEO,IE,F,26,2.748471e+09,0,False,77 days 08:00:16
151110,CMCXFGRHYSTVJ,207709,2015-07-09 20:06:07,2015-09-07 09:34:46,46,SEO,Chrome,M,37,3.601175e+09,0,True,59 days 13:28:39
